<a href="https://colab.research.google.com/github/Kiron-Ang/DFHL/blob/main/cortisol_depression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Print Python version and install/update/import libraries!
!python -V

!pip install -U scikit-learn > output.txt
import sklearn
print("scikit-learn", sklearn.__version__)

!pip install -U polars > output.txt
import polars
print("polars", polars.__version__)

!pip install -U matplotlib > output.txt
import matplotlib
print("matplotlib", matplotlib.__version__)

Python 3.10.12
scikit-learn 1.5.2
polars 1.9.0
matplotlib 3.9.2


In [2]:
# read_csv() to store data in a Polars DataFrame
cortisol = polars.read_csv("cortisol - cleansed.csv")
cortisol = cortisol.drop_nulls()
print("cortisol", cortisol.shape)
print("cortisol contains", cortisol.columns)

cortisol (21587, 14)
cortisol contains ['record_id', 'person_id', 'name', 'cortisol', 'month', 'day', 'year', 'collection', 'awakening', 'collection_minutes', 'awakening_minutes', 'difference_minutes', 'age', 'gender']


In [3]:
# Create two separate Polars DataFrames to hold input X and target y
X = cortisol["month", "collection_minutes", "awakening_minutes", "difference_minutes", "age", "gender"]
print("X", X.shape)
print("X contains", X.columns)
print(X)

y = cortisol["cortisol"]
print("y", y.shape)
print("y contains", y.name)
print(y)

X (21587, 6)
X contains ['month', 'collection_minutes', 'awakening_minutes', 'difference_minutes', 'age', 'gender']
shape: (21_587, 6)
┌───────┬────────────────────┬───────────────────┬────────────────────┬─────┬────────┐
│ month ┆ collection_minutes ┆ awakening_minutes ┆ difference_minutes ┆ age ┆ gender │
│ ---   ┆ ---                ┆ ---               ┆ ---                ┆ --- ┆ ---    │
│ i64   ┆ i64                ┆ i64               ┆ i64                ┆ i64 ┆ i64    │
╞═══════╪════════════════════╪═══════════════════╪════════════════════╪═════╪════════╡
│ 7     ┆ 450                ┆ 390               ┆ 60                 ┆ 1   ┆ 0      │
│ 8     ┆ 490                ┆ 420               ┆ 70                 ┆ 1   ┆ 0      │
│ 7     ┆ 465                ┆ 390               ┆ 75                 ┆ 1   ┆ 0      │
│ 7     ┆ 480                ┆ 375               ┆ 105                ┆ 1   ┆ 0      │
│ 7     ┆ 490                ┆ 360               ┆ 130                ┆ 1   ┆ 0   

In [4]:
# make_pipeline() to organize model steps
import sklearn.pipeline
import sklearn.ensemble

pipeline = sklearn.pipeline.make_pipeline(
    sklearn.ensemble.HistGradientBoostingRegressor(
        random_state = 900, scoring = "r2", validation_fraction = 0.2))

print(pipeline)

Pipeline(steps=[('histgradientboostingregressor',
                 HistGradientBoostingRegressor(random_state=900, scoring='r2',
                                               validation_fraction=0.2))])


In [5]:
"""
THIS IS IN A COMMENT BECAUSE GRIDSEARCH TAKES A LONG TIME TO EXECUTE
# GridSearchCV() to find the best parameters
import sklearn.model_selection

param_grid = {
    'histgradientboostingregressor__learning_rate': [0.001, 0.01, 0.1, 1],
    'histgradientboostingregressor__max_iter': [1, 10, 100, 1000],
    'histgradientboostingregressor__max_leaf_nodes': [10, 100, 1000, 10000],
}

gridsearchcv = sklearn.model_selection.GridSearchCV(pipeline, param_grid)

gridsearchcv.fit(X, y)

best_estimator = gridsearchcv.best_estimator_
print(best_estimator)

OUTPUT:
Pipeline(steps=[('histgradientboostingregressor',
                 HistGradientBoostingRegressor(learning_rate=0.01,
                                               max_iter=1000, max_leaf_nodes=10,
                                               random_state=900, scoring='r2',
                                               validation_fraction=0.2))])
"""

"\nTHIS IS IN A COMMENT BECAUSE GRIDSEARCH TAKES A LONG TIME TO EXECUTE\n# GridSearchCV() to find the best parameters\nimport sklearn.model_selection\n\nparam_grid = {\n    'histgradientboostingregressor__learning_rate': [0.001, 0.01, 0.1, 1],\n    'histgradientboostingregressor__max_iter': [1, 10, 100, 1000],\n    'histgradientboostingregressor__max_leaf_nodes': [10, 100, 1000, 10000],\n}\n\ngridsearchcv = sklearn.model_selection.GridSearchCV(pipeline, param_grid)\n\ngridsearchcv.fit(X, y)\n\nbest_estimator = gridsearchcv.best_estimator_\nprint(best_estimator)\n\nOUTPUT:\nPipeline(steps=[('histgradientboostingregressor',\n                 HistGradientBoostingRegressor(learning_rate=0.01,\n                                               max_iter=1000, max_leaf_nodes=10,\n                                               random_state=900, scoring='r2',\n                                               validation_fraction=0.2))])\n"

In [6]:
# make_pipeline() to incorporate grid search results
import sklearn.pipeline
import sklearn.ensemble

pipeline = sklearn.pipeline.make_pipeline(
    sklearn.ensemble.HistGradientBoostingRegressor(
        random_state = 900, scoring = "r2", validation_fraction = 0.2,
        learning_rate = 0.01, max_iter = 1000, max_leaf_nodes = 10))

print(pipeline)

Pipeline(steps=[('histgradientboostingregressor',
                 HistGradientBoostingRegressor(learning_rate=0.01,
                                               max_iter=1000, max_leaf_nodes=10,
                                               random_state=900, scoring='r2',
                                               validation_fraction=0.2))])


In [7]:
# cross_validate() to determine model performance
cross_validate = sklearn.model_selection.cross_validate(
    pipeline, X, y)

print("r^2 score:", cross_validate["test_score"].mean())

r^2 score: 0.415798788794548


In [8]:
# fit() on all data
pipeline.fit(X, y)

# read_csv() to store data in a Polars DataFrame
bdi = polars.read_csv("cortisol - 402_people.csv")
bdi = bdi.drop_nulls()
print("bdi", bdi.shape)
print("bdi contains", bdi.columns)

bdi (402, 5)
bdi contains ['id', 'name', 'gender', 'birth_year', 'bdi']


In [9]:
# Extrapolate two cortisol measurements with predict():
# 1) 7 AM in July 2003
# 2) 4 PM in July 2003

# Add age column; age = 2003 - birth_year
age = polars.Series("age", 2003 - bdi["birth_year"])
bdi.insert_column(-1, age)

# Add month column; BDI survey scores were collected in July, 2003
month = polars.Series("month", [7] * 402)
bdi.insert_column(-1, month)

# Add awakening_minutes column; assume everyone wakes up at 6 AM
awakening_minutes = polars.Series("awakening_minutes", [360] * 402)
bdi.insert_column(-1, awakening_minutes)

# Add collection_minutes_1; assume 7 AM
collection_minutes_1 = polars.Series("collection_minutes_1", [420] * 402)
bdi.insert_column(-1, collection_minutes_1)

# Add collection_minutes_2; assume 4 PM
collection_minutes_2 = polars.Series("collection_minutes_2", [960] * 402)
bdi.insert_column(-1, collection_minutes_2)

# Add difference_minutes_1
difference_minutes_1 = polars.Series("difference_minutes_1", bdi["collection_minutes_1"] - bdi["awakening_minutes"])
bdi.insert_column(-1, difference_minutes_1)

# Add difference_minutes_2
difference_minutes_2 = polars.Series("difference_minutes_2", bdi["collection_minutes_2"] - bdi["awakening_minutes"])
bdi.insert_column(-1, difference_minutes_2)

print("bdi", bdi.shape)
print("bdi contains", bdi.columns)

bdi (402, 12)
bdi contains ['id', 'name', 'gender', 'birth_year', 'age', 'month', 'awakening_minutes', 'collection_minutes_1', 'collection_minutes_2', 'difference_minutes_1', 'difference_minutes_2', 'bdi']


In [10]:
# Create new two new inputs in Polars DataFrames, X_1, X_2, to
# predict cortisol measurements, cortisol_1, cortisol_2
X_1 = bdi["month", "collection_minutes_1", "awakening_minutes", "difference_minutes_1", "age", "gender"]
print("X_1", X_1.shape)
print("X_1 contains", X_1.columns)
print(X_1)

X_2 = bdi["month", "collection_minutes_2", "awakening_minutes", "difference_minutes_2", "age", "gender"]
print("X_2", X_2.shape)
print("X_2 contains", X_2.columns)
print(X_2)

X_1 (402, 6)
X_1 contains ['month', 'collection_minutes_1', 'awakening_minutes', 'difference_minutes_1', 'age', 'gender']
shape: (402, 6)
┌───────┬──────────────────────┬───────────────────┬──────────────────────┬─────┬────────┐
│ month ┆ collection_minutes_1 ┆ awakening_minutes ┆ difference_minutes_1 ┆ age ┆ gender │
│ ---   ┆ ---                  ┆ ---               ┆ ---                  ┆ --- ┆ ---    │
│ i64   ┆ i64                  ┆ i64               ┆ i64                  ┆ i64 ┆ i64    │
╞═══════╪══════════════════════╪═══════════════════╪══════════════════════╪═════╪════════╡
│ 7     ┆ 420                  ┆ 360               ┆ 60                   ┆ 7   ┆ 0      │
│ 7     ┆ 420                  ┆ 360               ┆ 60                   ┆ 14  ┆ 0      │
│ 7     ┆ 420                  ┆ 360               ┆ 60                   ┆ 11  ┆ 0      │
│ 7     ┆ 420                  ┆ 360               ┆ 60                   ┆ 10  ┆ 0      │
│ 7     ┆ 420                  ┆ 360       